In [ ]:
import numpy as np
from scipy.optimize import fsolve, root
import matplotlib.pyplot as plt

# --- Global Epsilon for Numerical Stability ---
# A small positive number for numerical stability, used to avoid log(0) or division by zero.
epsilon_val = 1e-10

# 1. Parameters Definition
#    Based on GHKT (Golosov et al., 2014) and Barrage (2014) for the base model,
#    and Chazel et al. (2023) for mineral-specific parameters.

class Parameters:
    def __init__(self):
        # Model Time Step
        self.time_step_length = 10 # Years per period (e.g., 10 for decadal)

        # GHKT Baseline Parameters (from GHKT 2014 Table 1 and Barrage 2014 Table S-I)
        self.beta_annual_base = 0.985 # Annual base discount factor (GHKT's decadal beta 0.985^10 comes from this annual rate)
        self.beta = self.beta_annual_base**self.time_step_length # Actual decadal beta used in model periods

        self.alpha = 0.3          # Output elasticity of capital
        self.nu = 0.04            # Output elasticity of energy
        self.gamma = 2.3793e-5    # Damage parameter (gamma_bar in GHKT)
        self.phi_L = 0.2          # Share of carbon remaining permanently
        self.phi_0 = 0.393        # Share of remaining emissions after immediate absorption
        self.epsilon_annual_base = 1 - 0.0228 # Annual base decay factor for atmospheric carbon (1 - phi from GHKT)
        self.epsilon = self.epsilon_annual_base**self.time_step_length # Actual decadal decay factor

        self.rho_energy = -0.058  # Parameter for interfuel substitution
        self.kappa1 = 0.5429      # Share parameter for oil in energy composite
        self.kappa2 = 0.1015      # Share parameter for coal in energy composite
        self.kappa3 = 1 - self.kappa1 - self.kappa2 # Share parameter for green energy (calculated as 1 - kappa1 - kappa2)
        self.A2_0 = 7683          # Initial labor productivity in coal sector (A_2,0 in GHKT)
        self.A3_0 = 1311          # Initial labor productivity in green sector (A_3,0 in GHKT)
        self.omega2 = (1.02)**10 # Decadal growth rate for A2 (annual 1.02 compounded over 10 years)
        self.omega3 = (1.02)**10 # Decadal growth rate for A3 (annual 1.02 compounded over 10 years)
        self.S0_ghkt_oil_stock = 253.8 # Initial oil stock (GtC) in GHKT (R_0 in GHKT)
        self.N_0 = 1              # Total labor supply (N_0 in GHKT)
        self.Y0 = 700000          # Initial GDP per decade (Y_0 in GHKT)

        # GHKT Atmospheric Carbon Initial Conditions (from GHKT 2014 Table 1)
        self.S_total_initial = 802      # Total atmospheric carbon at time 0 (year 2000 in GHKT)
        self.S1_initial = 684           # Permanent component of atmospheric carbon at time 0
        self.S2_initial = self.S_total_initial - self.S1_initial # Transient component, calculated
        self.S_bar = 581                # Pre-industrial atmospheric CO2 concentration (GtC) (S_bar in GHKT)

        # Chazel et al. (2023) Specific Parameters (Table 3 in Chazel et al.)
        self.kappa_s = 0.3085     # Share parameter for secondary minerals in green capital
        self.kappa_p = 1 - self.kappa_s # Share parameter for primary minerals in green capital (calculated as 1-kappa_s)
        self.rho_green_capital = 0.5 # Substitution parameter for primary/secondary minerals in green capital
        self.Ap_0 = 132000        # Initial labor productivity in primary mineral extraction (A_p,0 in Chazel)
        self.As_0 = 132000        # Initial labor productivity in secondary mineral extraction (A_s,0 in Chazel)
        self.psi_green = 1.877    # Energy produced per unit of green capital (psi in Chazel)
        self.kappa_G = 0.75       # Share parameter for green capital in green energy production
        self.kappa_L = 1 - self.kappa_G # Share parameter for labor in green energy production (calculated as 1-kappa_G)
        self.rho_green_energy = -3 # Substitution parameter for labor/green capital in green energy production (rho_bar in Chazel)
        self.A3_0_chazel = 865.14 # Initial labor productivity in green energy sector (A_3,0 in Chazel)
        self.gA3_chazel = (1.02)**10 # Decadal growth rate for A3_chazel (annual 1.02 compounded over 10 years)
        self.gAs = (1.02)**10      # Decadal growth rate for As (annual 1.02 compounded over 10 years)
        self.gAp = (1.02)**10      # Decadal growth rate for Ap (annual 1.02 compounded over 10 years)
        self.Ms0 = 19             # Initial secondary mineral stock (MtCu) (M_s,0 in Chazel)
        self.Mp0_scenario = 2000 # Scenario for initial primary mineral stock (MtCu). Chazel examines 50, 200, 1000, 2000.
        self.T_horizon = 30       # Number of 10-year periods (300 years) as per Chazel et al. setup

        # Consistency checks (optional, but good practice)
        if not np.isclose(self.kappa1 + self.kappa2 + self.kappa3, 1):
            print("Warning: Kappa values for energy do not sum to 1. Please check parameters.")
        if not np.isclose(self.kappa_s + self.kappa_p, 1):
            print("Warning: Kappa values for green capital do not sum to 1. Please check parameters.")
        if not np.isclose(self.kappa_G + self.kappa_L, 1):
            print("Warning: Kappa values for green energy production do not sum to 1. Please check parameters.")

# Instantiate parameters outside the class definition to use them
params = Parameters()

# Optional: Print to verify (can remove or comment out later)
print("Model Parameters Loaded:")
for attr, value in vars(params).items():
    print(f"  {attr}: {type(value).__name__} = {value}")


# --- Global definitions for variable indices (used by full_system_equations and solve_model) ---
# These must be defined globally so solve_model can use them for initial guess and plotting.

# Control/Decision Variables (chosen by planner/market for each period t)
IDX_C = 0       # Consumption
IDX_N0 = 1      # Labor in final goods sector
IDX_LC = 2      # Labor in coal sector
IDX_N3 = 3      # Labor in green energy production (N3,t in Chazel)
IDX_Np = 4      # Labor in primary mineral extraction
IDX_Ns = 5      # Labor in secondary mineral extraction
IDX_F = 6       # Oil extraction (E1,t in GHKT)
IDX_mp = 7      # Primary mineral flow (m_p,t in Chazel)
IDX_ms = 8      # Secondary mineral flow (m_s,t in Chazel)

# State Variables (evolve over time, their values for t+1 are implicitly solved via transition equations)
# These are treated as variables to solve for across time.
IDX_Mu_oil = 9  # Oil scarcity rent
IDX_Mu_p = 10   # Primary mineral scarcity rent
IDX_Mu_s = 11   # Secondary mineral scarcity rent
IDX_Lambda_s = 12 # Optimal carbon tax (marginal externality damage)
IDX_S1 = 13     # Atmospheric Carbon Permanent Component (S1,t)
IDX_S2 = 14     # Atmospheric Carbon Transient Component (S2,t)
IDX_K = 15      # Capital Stock (K_t)
IDX_G = 16      # Green Capital Stock (G_t)
IDX_OilStock = 17 # Oil Stock (R_t in GHKT for oil)
IDX_MpStock = 18 # Primary Mineral Stock (M_p,t in Chazel)
IDX_MsStock = 19 # Secondary Mineral Stock (M_s,t in Chazel)

NUM_VARS_PER_PERIOD = 20 # Corrected to 20 to match highest IDX + 1


# 2.1. Household Preferences

def utility_function(C_t, N_t):
    """
    Household utility function, logarithmic in per capita consumption.
    (Assumption 1 in GHKT 2014, and Eq 1 in Chazel et al. 2023)
    C_t: Total consumption at time t
    N_t: Total labor supply/population at time t (N_t in GHKT and Chazel et al.)
    """
    # Ensure positive consumption and population to avoid log(0) or log(negative)
    if C_t <= epsilon_val or N_t <= epsilon_val:
        return -np.inf # Represents very low utility for invalid consumption

    return np.log(C_t / N_t)


# 2.2. Core Production Functions (Retained by Chazel et al. from GHKT Baseline)

def energy_composite_decadal(E1_t_oil, E2_t_coal, E3_t_green_decadal, params):
    """
    Energy composite function E_t for decadal periods.
    (Eq 21 in GHKT 2014)
    E1_t_oil: Oil input at time t (E1,t in GHKT)
    E2_t_coal: Coal input at time t (E2,t in GHKT)
    E3_t_green_decadal: Green energy input at time t (E3,t in GHKT)
    params: Parameters object.
    """
    # E_t = (kappa1*E1,t^rho + kappa2*E2,t^rho + kappa3*E3,t^rho)^(1/rho)
    # GHKT (2014) defines rho_energy = -0.058, which is a CES (Constant Elasticity of Substitution) form.
    if params.rho_energy == 0:
        # This case is theoretically Cobb-Douglas (rho=0, product form), but not applicable with GHKT's specified rho_energy.
        return (params.kappa1 * E1_t_oil**params.rho_energy *
                params.kappa2 * E2_t_coal**params.rho_energy *
                params.kappa3 * E3_t_green_decadal**params.rho_energy)**(1/params.rho_energy)
    else:
        return (params.kappa1 * E1_t_oil**params.rho_energy +
                params.kappa2 * E2_t_coal**params.rho_energy +
                params.kappa3 * E3_t_green_decadal**params.rho_energy)**(1/params.rho_energy)

def final_goods_output_decadal(K_t, N0_t, E_t_composite_decadal, S_t_total, A0_t_initial, params, t_period):
    """
    Final goods production function Y_t for decadal periods.
    (Eq 20 in GHKT 2014)
    K_t: Capital stock at time t
    N0_t: Labor allocated to final goods sector at time t
    E_t_composite_decadal: Composite energy input at time t
    S_t_total: Total atmospheric carbon stock at time t (S_t in GHKT notation)
    A0_t_initial: Initial total factor productivity in final goods sector (exogenous, A0 from GHKT)
    params: Parameters object.
    t_period: The current decadal period (0, 1, 2, ..., T_horizon-1)
    """
    # Y_t = exp(-gamma * (S_t - S_bar)) * A0,t * K_t^alpha * N0,t^(1-alpha-nu) * E_t^nu
    # GHKT assumes A0,t is constant in their benchmark for analytical solution.
    # Chazel uses A0 as a fixed value in Table 2 for GHKT parameters.
    
    A0_t = A0_t_initial # A0,t is constant in GHKT benchmark.

    damage_factor = np.exp(-params.gamma * (S_t_total - params.S_bar))
    return damage_factor * A0_t * (K_t**params.alpha) * \
           (N0_t**(1 - params.alpha - params.nu)) * (E_t_composite_decadal**params.nu)

# 2.3. Chazel et al. Specific Production Functions (Mineral & Green Capital)

def green_capital_stock(ms_t, mp_t, params):
    """
    Green capital production function G_t.
    (Eq 11 in Chazel et al. 2023)
    ms_t: Flow of secondary (recycled) minerals (m_s,t in Chazel)
    mp_t: Flow of primary minerals (m_p,t in Chazel)
    params: Parameters object.
    """
    # G_t = (kappa_s*m_s,t^rho_green_capital + kappa_p*m_p,t^rho_green_capital)^(1/rho_green_capital)
    # Chazel defines rho_green_capital = 0.5.
    if params.rho_green_capital == 0: # Cobb-Douglas case (product form), not applicable with Chazel's rho.
        return (params.kappa_s * ms_t**params.rho_green_capital *
                params.kappa_p * mp_t**params.rho_green_capital)**(1/params.rho_green_capital)
    else:
        return (params.kappa_s * ms_t**params.rho_green_capital +
                params.kappa_p * mp_t**params.rho_green_capital)**(1/params.rho_green_capital)

def green_energy_production_chazel_decadal(A3_0_chazel_initial, gA3_chazel_growth_rate, t_period, N3_t, G_t, params):
    """
    Low-carbon energy production function E3_t.
    (Eq 10 in Chazel et al. 2023)
    A3_0_chazel_initial: Initial labor productivity in green energy sector (A_3,0 in Chazel)
    gA3_chazel_growth_rate: Decadal growth rate for A3_chazel (g_A3 in Chazel)
    t_period: The current decadal period (0, 1, 2, ...)
    N3_t: Labor allocated to green energy production (N3,t in Chazel)
    G_t: Green capital stock at time t (G_t in Chazel)
    params: Parameters object.
    """
    A3_t_chazel = A3_0_chazel_initial * (gA3_chazel_growth_rate**t_period)
    
    # E3_t = [kappa_L*(A3,t*N3,t)^rho_green_energy + kappa_G*(psi_green*G_t)^rho_green_energy]^(1/rho_green_energy)
    # Chazel defines rho_green_energy = -3.
    if params.rho_green_energy == 0: # Cobb-Douglas case (product form), not applicable with Chazel's rho.
        return (params.kappa_L * (A3_t_chazel * N3_t)**params.rho_green_energy *
                params.kappa_G * (params.psi_green * G_t)**params.rho_green_energy)**(1/params.rho_green_energy)
    else:
        return (params.kappa_L * (A3_t_chazel * N3_t)**params.rho_green_energy +
                params.kappa_G * (params.psi_green * G_t)**params.rho_green_energy)**(1/params.rho_green_energy)

# 3. State Variables and their Evolution

def update_capital_decadal(Y_t, C_t):
    """
    Capital accumulation for decadal periods.
    (Based on GHKT 2014 Eq 2 and Chazel et al. 2023 discussion of full depreciation over a decade)
    Y_t: Total output at time t
    C_t: Total consumption at time t
    """
    # GHKT (2014) and Chazel et al. (2023) assume full depreciation: K_t+1 = Y_t - C_t.
    investment = Y_t - C_t
    
    # Ensure non-negative capital. If investment is negative, capital can decrease.
    # The original model structure doesn't typically enforce K_t+1 >= 0 explicitly in this simple form,
    # but the solver would implicitly ensure feasible paths.
    return investment

def update_atmospheric_carbon_states_decadal(S1_prev, S2_prev, E_fossil_t, params):
    """
    Updates permanent (S1) and transient (S2) atmospheric carbon stocks for decadal periods.
    (Eq 16 in Chazel et al. 2023, derived from GHKT 2014 carbon cycle)
    S1_prev: Permanent carbon stock from previous period (S1,t-1)
    S2_prev: Transient carbon stock from previous period (S2,t-1)
    E_fossil_t: Total fossil fuel emissions at time t (Oil + Coal in carbon content units)
    params: Parameters object.
    """
    # S1,t = S1,t-1 + phi_L * E_fossil,t
    # (S2,t - S_bar) = epsilon_decadal * (S2,t-1 - S_bar) + phi_0_decadal * (1-phi_L) * E_fossil,t
    
    S1_next = S1_prev + params.phi_L * E_fossil_t
    S2_next = params.epsilon * (S2_prev - params.S_bar) + params.S_bar + params.phi_0 * (1 - params.phi_L) * E_fossil_t
    return S1_next, S2_next

def update_mineral_stocks_decadal(Mp_t_prev, ms_t, mp_t, Ms_t_prev):
    """
    Updates primary (Mp) and secondary (Ms) mineral stocks for decadal periods.
    (Eq 14 and 17 in Chazel et al. 2023)
    Mp_t_prev: Primary mineral stock from previous period (R_i,t in GHKT for general resource, Mp_t in Chazel)
    ms_t: Flow of secondary (recycled) minerals at time t (m_s,t in Chazel)
    mp_t: Flow of primary minerals at time t (m_p,t in Chazel)
    Ms_t_prev: Secondary mineral stock from previous period (Ms_t in Chazel)
    """
    # Mp,t+1 = Mp,t - m_p,t
    # M_s,t+1 = M_s,t + m_p,t (simplified from M_s,t - m_s,t + m_s,t + m_p,t)
    
    Mp_next = Mp_t_prev - mp_t
    Ms_next = Ms_t_prev + mp_t 

    return Mp_next, Ms_next


# 4. First Order Conditions (FOCs) and System Equations for Solver

def full_system_equations(variables_flat, params):
    """
    Defines the full system of non-linear equations (FOCs and state transitions)
    to be solved simultaneously for all time steps.

    Args:
        variables_flat (np.ndarray): A 1D array of all variables (control and state) for all time steps.
        params (Parameters): Parameters object.

    Returns:
        np.ndarray: A 1D array of residuals, where each element should be zero at the solution.
    """

    T = params.T_horizon # Total number of periods

    # Reshape the flattened input array into a 2D array (T periods x NUM_VARS_PER_PERIOD)
    variables = variables_flat.reshape(T, NUM_VARS_PER_PERIOD)
    
    # Initialize list to store residuals (equations to be satisfied)
    residuals = []

    # --- Initial State Variable Conditions (at t=0) ---
    # These fix the first period's state variables to their calibrated initial values.
    # They serve as equations that must be satisfied by the solver for t=0.

    # Residual for initial capital stock K[0]
    # GHKT (2014) provides analytical solution for K_0 = alpha * beta * Y_0 / (1 - beta * (1 - alpha))
    # Barrage (2014) Table S-II gives K0 = 128920.
    residuals.append(variables[0, IDX_K] - 128920)

    # Residual for initial atmospheric carbon S1[0]
    residuals.append(variables[0, IDX_S1] - params.S1_initial)

    # Residual for initial atmospheric carbon S2[0]
    residuals.append(variables[0, IDX_S2] - params.S2_initial)

    # Residual for initial oil stock (R_0 in GHKT 2014)
    # The initial amount of oil R0 (GtC) is 253.8.
    residuals.append(variables[0, IDX_OilStock] - params.S0_ghkt_oil_stock) 

    # Residual for initial primary mineral stock Mp[0]
    # Chazel et al. (2023) models four copper budget scenarios for wind energy production, ranging from Mp,0=50MtCu to Mp,0=2000MtCu.
    # We use 2000 for params.Mp0_scenario.
    residuals.append(variables[0, IDX_MpStock] - params.Mp0_scenario)

    # Residual for initial secondary mineral stock Ms[0]
    # Chazel et al. (2023) assumes that the initial secondary mineral stock is 19Mt copper.
    residuals.append(variables[0, IDX_MsStock] - params.Ms0)
    
    # Residual for initial green capital G[0]
    # Chazel et al. (2023) does not explicitly state G0. We assume 0 for simplicity if not given.
    residuals.append(variables[0, IDX_G] - 0)

    # --- Loop through time periods to define equations (t from 0 to T-1) ---
    for t in range(T):
        # Extract variables for current period (t)
        C_t = variables[t, IDX_C]
        N0_t = variables[t, IDX_N0]
        LC_t = variables[t, IDX_LC]
        N3_t = variables[t, IDX_N3]
        Np_t = variables[t, IDX_Np]
        Ns_t = variables[t, IDX_Ns]
        F_t = variables[t, IDX_F]
        mp_t = variables[t, IDX_mp]
        ms_t = variables[t, IDX_ms]
        Mu_oil_t = variables[t, IDX_Mu_oil]
        Mu_p_t = variables[t, IDX_Mu_p]
        Mu_s_t = variables[t, IDX_Mu_s]
        Lambda_s_t = variables[t, IDX_Lambda_s]
        S1_t = variables[t, IDX_S1]
        S2_t = variables[t, IDX_S2]
        K_t = variables[t, IDX_K]
        G_t = variables[t, IDX_G]
        OilStock_t = variables[t, IDX_OilStock]
        MpStock_t = variables[t, IDX_MpStock]
        MsStock_t = variables[t, IDX_MsStock]

        # Helper for numerical stability (avoid division by zero or log(0)).
        # Apply np.maximum(val, epsilon_val) where values might go to zero or negative.
        # These are defined unconditionally at the start of each loop iteration.
        C_t_safe = np.maximum(C_t, epsilon_val)
        K_t_safe = np.maximum(K_t, epsilon_val)
        N0_t_safe = np.maximum(N0_t, epsilon_val)
        LC_t_safe = np.maximum(LC_t, epsilon_val)
        N3_t_safe = np.maximum(N3_t, epsilon_val)
        Np_t_safe = np.maximum(Np_t, epsilon_val)
        Ns_t_safe = np.maximum(Ns_t, epsilon_val)
        F_t_safe = np.maximum(F_t, epsilon_val)
        mp_t_safe = np.maximum(mp_t, epsilon_val)
        ms_t_safe = np.maximum(ms_t, epsilon_val)
        G_t_safe = np.maximum(G_t, epsilon_val)
        OilStock_t_safe = np.maximum(OilStock_t, epsilon_val)
        MpStock_t_safe = np.maximum(MpStock_t, epsilon_val)
        MsStock_t_safe = np.maximum(MsStock_t, epsilon_val)
        S_t_total_safe = np.maximum(S1_t + S2_t, epsilon_val)


        # Initialize _plus_1 variables with default 0s. These will be overwritten for t < T-1.
        # For t = T-1, these will remain 0, aligning with implicit terminal conditions for these
        # 'next period' references if not explicitly set by a terminal condition in 4.6.
        # These are primarily for the variables that are *solved for* in the next period.
        C_t_plus_1 = 0
        Mu_oil_t_plus_1 = 0
        Mu_p_t_plus_1 = 0
        Mu_s_t_plus_1 = 0
        Lambda_s_t_plus_1 = 0
        S1_t_plus_1 = 0
        S2_t_plus_1 = 0
        K_t_plus_1 = 0
        G_t_plus_1 = 0
        OilStock_t_plus_1 = 0
        MpStock_t_plus_1 = 0
        MsStock_t_plus_1 = 0
        
        # Productivities for next period (based on decadal growth rates) also need initialization for t=T-1 case
        A2_t_plus_1 = 0
        A3_t_plus_1 = 0
        Ap_t_plus_1 = 0
        As_t_plus_1 = 0
        A3_t_chazel_plus_1 = 0


        # Extract actual _plus_1 variables if t is not the last period (t < T-1)
        if t < T - 1:
            C_t_plus_1 = variables[t+1, IDX_C]
            Mu_oil_t_plus_1 = variables[t+1, IDX_Mu_oil]
            Mu_p_t_plus_1 = variables[t+1, IDX_Mu_p]
            Mu_s_t_plus_1 = variables[t+1, IDX_Mu_s]
            Lambda_s_t_plus_1 = variables[t+1, IDX_Lambda_s]
            S1_t_plus_1 = variables[t+1, IDX_S1]
            S2_t_plus_1 = variables[t+1, IDX_S2]
            K_t_plus_1 = variables[t+1, IDX_K]
            G_t_plus_1 = variables[t+1, IDX_G]
            OilStock_t_plus_1 = variables[t+1, IDX_OilStock]
            MpStock_t_plus_1 = variables[t+1, IDX_MpStock]
            MsStock_t_plus_1 = variables[t+1, IDX_MsStock]
            
            # Next period's productivities (based on decadal growth rates)
            A2_t_plus_1 = params.A2_0 * (params.omega2**(t + 1))
            A3_t_plus_1 = params.A3_0 * (params.omega3**(t + 1))
            Ap_t_plus_1 = params.Ap_0 * (params.gAp**(t + 1))
            As_t_plus_1 = params.As_0 * (params.gAs**(t + 1))
            A3_t_chazel_plus_1 = params.A3_0_chazel * (params.gA3_chazel**(t + 1))


        # Derived intermediate variables using safe versions for current period (t)
        A0_t = 17887 # A0 is constant as per Barrage (2014) Table S-II for calibration.
        A2_t = params.A2_0 * (params.omega2**t)
        A3_t = params.A3_0 * (params.omega3**t)
        Ap_t = params.Ap_0 * (params.gAp**t)
        As_t = params.As_0 * (params.gAs**t)
        A3_t_chazel = params.A3_0_chazel * (params.gA3_chazel**t)
        
        S_t_total = S1_t + S2_t # Total atmospheric carbon at current period
        N_t = params.N_0 # Total labor is constant in GHKT/Chazel baseline

        # Calculate energy composite and final output using current period values
        E3_t = green_energy_production_chazel_decadal(A3_t_chazel, params.gA3_chazel, t, N3_t_safe, G_t_safe, params)
        E3_t_safe = np.maximum(E3_t, epsilon_val)
        
        E_fossil_t = F_t_safe + (A2_t * LC_t_safe) # Oil extraction + Coal production (sum in GtC)
        
        E_t_composite = energy_composite_decadal(F_t_safe, (A2_t * LC_t_safe), E3_t_safe, params)
        E_t_composite_safe = np.maximum(E_t_composite, epsilon_val)
        
        Y_t = final_goods_output_decadal(K_t_safe, N0_t_safe, E_t_composite_safe, S_t_total_safe, A0_t, params, t)
        Y_t_safe = np.maximum(Y_t, epsilon_val)


        # --- Equations for each period t (residuals to be zero) ---

        # 1. Resource Constraint (Consumption + next Capital = Current Output)
        # GHKT (2014) Eq 2: C_t + K_t+1 = Y_t (simplified for full depreciation from GHKT Sec 3).
        if t < T - 1:
            residuals.append(C_t + K_t_plus_1 - Y_t) 

        # 2. Labor Constraint (Total Labor = Sum of Labor Allocations)
        # Chazel (2023) Eq 18: N0,t + N2,t + N3,t + Np,t + Ns,t = N_t.
        # Here N2,t is LC_t, N3,t is N3_t, N_t is L_t.
        residuals.append(N0_t + LC_t + N3_t + Np_t + Ns_t - N_t)

        # 3. Capital Accumulation (K_t+1 update rule)
        # K_t+1 is implicitly defined by the resource constraint (Eq 1). We ensure the K_t+1 variable matches.
        # This is already handled by Eq 1 (C_t + K_t+1 - Y_t = 0).

        # 4. Atmospheric Carbon S1/S2 Accumulation (Chazel 2023 Eq 16)
        if t < T - 1:
            S1_next_derived, S2_next_derived = update_atmospheric_carbon_states_decadal(S1_t_safe, S2_t_safe, E_fossil_t, params)
            residuals.append(S1_t_plus_1 - S1_next_derived)
            residuals.append(S2_t_plus_1 - S2_next_derived)

        # 5. Oil Stock Update (GHKT 2014 Eq 1: R_i,t+1 = R_i,t - E_i,t for general resource)
        if t < T - 1:
            residuals.append(OilStock_t_plus_1 - (OilStock_t_safe - F_t_safe))

        # 6. Primary Mineral Stock Update (Chazel 2023 Eq 14: M_p,t+1 = M_p,t - m_p,t)
        if t < T - 1:
            residuals.append(MpStock_t_plus_1 - (MpStock_t_safe - mp_t_safe))

        # 7. Secondary Mineral Stock Update (Chazel 2023 Eq 17: M_s,t+1 = M_s,t + m_p,t)
        if t < T - 1:
            residuals.append(MsStock_t_plus_1 - (MsStock_t_safe + mp_t_safe))
        
        # 8. Green Capital Stock Definition (Chazel 2023 Eq 11: G_t = (kappa_s*m_s,t^rho + kappa_p*m_p,t^rho)^(1/rho))
        residuals.append(G_t - green_capital_stock(ms_t_safe, mp_t_safe, params))


        # --- 4.5. First Order Conditions (FOCs) ---

        # FOC 9: Euler Equation for Consumption (GHKT 2014)
        # 1/C_t = beta * (1/C_t+1) * (1 + MPK_t+1)
        # This is for t < T-1.
        if t < T - 1:
            # Need Y_t_plus_1 and E_t_composite_plus_1 for MPK.
            E3_t_plus_1_derived = green_energy_production_chazel_decadal(A3_t_chazel_plus_1, params.gA3_chazel, t+1, variables[t+1, IDX_N3], G_t_plus_1, params)
            E3_t_plus_1_safe = np.maximum(E3_t_plus_1_derived, epsilon_val)
            E_t_composite_plus_1_derived = energy_composite_decadal(variables[t+1, IDX_F], (A2_t_plus_1 * variables[t+1, IDX_LC]), E3_t_plus_1_safe, params)
            E_t_composite_plus_1_safe = np.maximum(E_t_composite_plus_1_derived, epsilon_val)

            # Important: S_t_plus_1_total_safe is correctly derived from variables[t+1, IDX_S1/S2]
            S_t_plus_1_total_safe = np.maximum(variables[t+1, IDX_S1] + variables[t+1, IDX_S2], epsilon_val)
            
            Y_t_plus_1_derived = final_goods_output_decadal(K_t_plus_1, variables[t+1, IDX_N0], E_t_composite_plus_1_safe, S_t_plus_1_total_safe, A0_t, params, t+1)
            Y_t_plus_1_safe = np.maximum(Y_t_plus_1_derived, epsilon_val)

            MPK_t_plus_1 = params.alpha * Y_t_plus_1_safe / K_t_plus_1 # Marginal product of capital (MPK)
            
            # Euler Equation residual
            residuals.append((C_t_plus_1 / C_t_safe) * (1 + MPK_t_plus_1) - (1 / params.beta))

        # FOC 10: Labor Allocation for Final Goods (N0_t) vs. Coal (LC_t)
        # MPN0 = (MPNC - Lambda_s_t) (Chazel Eq 19)
        MPN0 = (1 - params.alpha - params.nu) * Y_t_safe / N0_t_safe

        MPNC_pre_tax = (params.nu * params.kappa2 * (A2_t * LC_t_safe)**(params.rho_energy - 1) * A2_t) / \
                       (E_t_composite_safe**(params.rho_energy - 1) * LC_t_safe) * Y_t_safe / E_t_composite_safe
        
        residuals.append(MPN0 - (MPNC_pre_tax - Lambda_s_t))

        # FOC 11: Labor Allocation for Final Goods (N0_t) vs. Green Energy (N3_t)
        # MPN0 = MPN3 (Chazel Eq 20)
        MPN3_raw_dE3_dN3 = (params.kappa_L * (A3_t_chazel * N3_t_safe)**(params.rho_green_energy - 1) * A3_t_chazel) / \
                           (E3_t_safe**(params.rho_green_energy - 1))
        
        MPN3_converted = MPN3_raw_dE3_dN3 * (params.nu / E_t_composite_safe) * \
                         (E_t_composite_safe / E3_t_safe)**(1 - params.rho_energy) * Y_t_safe
        
        residuals.append(MPN0 - MPN3_converted)

        # FOC 12: Labor Allocation for Final Goods (N0_t) vs. Secondary Mineral Extraction (Ns_t)
        # MPN0 = MPNs (Chazel Eq 21)
        MPNs_raw_dG_dms = (params.kappa_s * ms_t_safe**(params.rho_green_capital - 1)) / \
                          (G_t_safe**(params.rho_green_capital - 1))
        MPNs_raw_dms_dNs = As_t # d(ms)/d(Ns) = As_t (from Eq 15 Chazel: m_s,t = A_s,t * N_s,t)
        
        MPNs_converted = MPNs_raw_dG_dms * MPNs_raw_dms_dNs * \
                         (params.kappa_G * params.psi_green**(params.rho_green_energy)) * \
                         (E3_t_safe**(params.rho_green_energy - 1)) / (A3_t_chazel * N3_t_safe)**(params.rho_green_energy - 1) * \
                         (E3_t_safe / G_t_safe)**(1 - params.rho_green_energy) * \
                         (params.nu / E_t_composite_safe) * (E_t_composite_safe / E3_t_safe)**(1 - params.rho_energy) * Y_t_safe
        
        residuals.append(MPN0 - MPNs_converted)

        # FOC 13: Hotelling Rule for Oil (GHKT 2014 Eq 22, Chazel 2023 Prop 2 Eq 22)
        # (nu * kappa1 / (E1,t^(1-rho_energy) * E_t_composite^rho_energy)) - Lambda_s_t = beta * (next period terms)
        # This is for t < T-1.
        if t < T - 1:
            MP_oil_current_pre_tax = (params.nu * params.kappa1) / \
                                     (F_t_safe**(1 - params.rho_energy) * E_t_composite_safe**params.rho_energy)
            
            # Recalculate E_t_composite for next period with SAFE values from next_period_variables.
            E_t_composite_plus_1_derived = energy_composite_decadal(variables[t+1, IDX_F], (A2_t_plus_1 * variables[t+1, IDX_LC]), green_energy_production_chazel_decadal(A3_t_chazel_plus_1, params.gA3_chazel, t+1, variables[t+1, IDX_N3], variables[t+1, IDX_G], params), params)
            E_t_composite_plus_1_safe = np.maximum(E_t_composite_plus_1_derived, epsilon_val)

            MP_oil_next_pre_tax = (params.nu * params.kappa1) / \
                                  (variables[t+1, IDX_F]**(1 - params.rho_energy) * E_t_composite_plus_1_safe**params.rho_energy)
            
            # Hotelling rule residual: (Current_Net_Marginal_Value) - (Discounted_Next_Period_Net_Marginal_Value) = 0
            residuals.append((MP_oil_current_pre_tax - Lambda_s_t) - (params.beta * (MP_oil_next_pre_tax - Lambda_s_t_plus_1)))

        # FOC 14: Hotelling Rule for Primary Mineral (Chazel 2023 Prop 2 Eq 23)
        # [kappa_p * Pi_G,t+1 * (G_t+1/mp_t+1)^(1-rho_green_capital) + (Y_t+1/C_t+1) * (1-alpha-nu)/(Ap_t+1 * N0_t+1)] = 1/beta * [current terms]
        if t < T - 1:
            # Terms for current period
            TermB_current = (Y_t_safe / C_t_safe) * (1 - params.alpha - params.nu) / (Ap_t * N0_t_safe)

            # Terms for next period (derived using next period variables)
            TermB_next_derived = (Y_t_plus_1_safe / C_t_plus_1) * (1 - params.alpha - params.nu) / (Ap_t_plus_1 * variables[t+1, IDX_N0])

            # Pi_G,t (Marginal Contribution of Green Capital to Welfare)
            # Chazel page 17, Pi_G,t = -(Y_t/C_t) * nu*kappa3*kappaG*psi / E_t * (E_t/E3_t)^(1-rho_energy) * (E3_t/(psi_green*G_t))^(1-rho_green_energy)
            Pi_G_current = - (Y_t_safe / C_t_safe) * (params.nu * params.kappa3 * params.kappa_G * params.psi_green) / E_t_composite_safe * \
                             (E_t_composite_safe / E3_t_safe)**(1 - params.rho_energy) * \
                             (E3_t_safe / (params.psi_green * G_t_safe))**(1 - params.rho_green_energy)
            
            # Pi_G,t+1 (Derived for next period)
            E3_t_plus_1_derived = green_energy_production_chazel_decadal(A3_t_chazel_plus_1, params.gA3_chazel, t+1, variables[t+1, IDX_N3], G_t_plus_1, params)
            E3_t_plus_1_safe = np.maximum(E3_t_plus_1_derived, epsilon_val)
            E_t_composite_plus_1_derived = energy_composite_decadal(variables[t+1, IDX_F], (A2_t_plus_1 * variables[t+1, IDX_LC]), E3_t_plus_1_safe, params)
            E_t_composite_plus_1_safe = np.maximum(E_t_composite_plus_1_derived, epsilon_val)
            
            # Use correct S_t_plus_1_total_safe for Y_t_plus_1_derived calculation
            S_t_plus_1_total_safe = np.maximum(variables[t+1, IDX_S1] + variables[t+1, IDX_S2], epsilon_val)
            Y_t_plus_1_derived = final_goods_output_decadal(K_t_plus_1, variables[t+1, IDX_N0], E_t_composite_plus_1_safe, S_t_plus_1_total_safe, A0_t, params, t+1)
            Y_t_plus_1_safe = np.maximum(Y_t_plus_1_derived, epsilon_val)
            
            Pi_G_next_derived = - (Y_t_plus_1_safe / C_t_plus_1) * (params.nu * params.kappa3 * params.kappa_G * params.psi_green) / E_t_composite_plus_1_safe * \
                                 (E_t_composite_plus_1_safe / E3_t_plus_1_safe)**(1 - params.rho_energy) * \
                                 (E3_t_plus_1_safe / (params.psi_green * G_t_plus_1))**(1 - params.rho_green_energy)

            # TermA depends on Pi_G,t
            TermA_current = params.kappa_p * Pi_G_current * (G_t_safe / mp_t_safe)**(1 - params.rho_green_capital)
            mp_t_plus_1_safe = np.maximum(variables[t+1, IDX_mp], epsilon_val)
            TermA_next_derived = params.kappa_p * Pi_G_next_derived * (G_t_plus_1 / mp_t_plus_1_safe)**(1 - params.rho_green_capital)

            # Hotelling rule residual
            residuals.append((TermA_current + TermB_current) - (1 / params.beta) * (TermA_next_derived + TermB_next_derived))


        # FOC 15: Hotelling Rule for Secondary Mineral (Chazel 2023 Prop 2 for Lambda_s)
        # Note: Chazel's Prop 2, Eq 23, includes Mu_s. The text of P2 says "Hotelling's rule applies to the resource scarcity rents... which increase at a rate equals to the discount rate."
        # This is a complex derivation in Chazel, so we will use the simplified structure.
        # Chazel Proposition 2 doesn't give a separate Mu_s rule, it's combined with Mu_p.
        # However, the structure in the Mathematica notebook (Mu_s_t = beta * Mu_s_t+1) is a common simplification if cost of extraction depends only on labor.
        # Assuming simplified Hotelling for Mu_s similar to Mu_oil.
        if t < T - 1:
            residuals.append(Mu_s_t - params.beta * Mu_s_t_plus_1)


        # FOC 16: Carbon Tax (Marginal Externality Damage) FOC (GHKT 2014 Prop 1)
        # Lambda_s_t = Y_t * E_t[ sum_j (beta^j * gamma_t+j * (1-d_j)) ]
        # In GHKT/Barrage/Chazel, Lambda_s_t / Y_t is a constant (given constant gamma, beta, phi_L, phi_0, epsilon).
        # Lambda_s_t / Y_t = gamma_bar * (phi_L/(1-beta) + (1-phi_L)*phi_0/(1-(1-phi)*beta))
        
        # We need to ensure Lambda_s_t (variable) matches this analytical expression.
        const_factor = (params.phi_L / (1 - params.beta)) + \
                       ((1 - params.phi_L) * params.phi_0 / (1 - (1 - params.epsilon) * params.beta))
        
        Lambda_s_analytical = params.gamma * const_factor * Y_t_safe
        
        residuals.append(Lambda_s_t - Lambda_s_analytical)


    # --- 4.6. Terminal Conditions (for the last period T-1, specifically for variables at T) ---
    # These ensure consistency at the end of the simulation horizon.
    # Typically set scarcity rents of exhaustible resources to 0 if they are exhausted by the end.
    # Or enforce final stock values.

    # We will assume that all scarcity rents (Mu_oil, Mu_p, Mu_s) are zero at the very last period (T-1).
    # This implies the resources are optimally depleted by the end of the horizon.
    # For a total of T periods (0 to T-1), the last period is T-1.
    # The variables at T-1 represent the state *at the end* of the simulation.
    # So, we set the scarcity rents for the last period (T-1) to be 0.

    # Terminal Scarcity Rent for Oil
    residuals.append(variables[T-1, IDX_Mu_oil] - 0)

    # Terminal Scarcity Rent for Primary Mineral
    residuals.append(variables[T-1, IDX_Mu_p] - 0)

    # Terminal Scarcity Rent for Secondary Mineral
    residuals.append(variables[T-1, IDX_Mu_s] - 0)


    return np.array(residuals) # Return the flattened array of residuals


# 5. Numerical Solver Setup and Execution

def solve_model(params, initial_guess_flat=None):
    """
    Sets up and runs the numerical solver for the dynamic general equilibrium model.

    Args:
        params (Parameters): Parameters object containing model parameters.
        initial_guess_flat (np.ndarray, optional): A 1D array of initial guesses for all variables
                                                  across all time periods. If None, a default
                                                  linear guess is generated.

    Returns:
        tuple: A tuple containing:
               - result (scipy.optimize.OptimizeResult): The result object from the solver.
               - solution_reshaped (np.ndarray): The solved variables reshaped into a 2D array
                                                 (T_horizon x NUM_VARS_PER_PERIOD).
    """

    T = params.T_horizon
    
    # --- Define NUM_VARS_PER_PERIOD for consistency with full_system_equations ---
    # This must match the index mapping in full_system_equations exactly.
    NUM_VARS_PER_PERIOD = 20 # As defined in full_system_equations

    # --- Generate Initial Guess if not provided ---
    if initial_guess_flat is None:
        print("Generating default initial guess...")
        # Create a plausible initial guess for all variables across all periods (T).
        # This is CRUCIAL for solver convergence. A bad guess will lead to non-convergence.

        # Initialize a 2D array for variables
        # We need a starting point for each of the NUM_VARS_PER_PERIOD variables for each of the T periods.
        initial_guess_2d = np.zeros((T, NUM_VARS_PER_PERIOD))

        # Base values from params (decadal scale)
        initial_K0 = 128920
        initial_S1 = params.S1_initial
        initial_S2 = params.S2_initial
        initial_OilStock = params.S0_ghkt_oil_stock
        initial_MpStock = params.Mp0_scenario
        initial_MsStock = params.Ms0
        initial_G = 0 # Assumed G0

        # Target values (simplified assumptions for a guess - could be steady state or depletion)
        # These are highly speculative for an initial guess, but aim to guide the solver.
        final_K = initial_K0 * 0.5 # Assume capital might decline a bit
        final_OilStock = epsilon_val # Close to zero for depletion
        final_MpStock = epsilon_val # Close to zero for depletion
        final_MsStock = initial_MsStock * 5 # Assume secondary minerals grow
        final_S1 = initial_S1 * 1.5 # Assume S1 might increase due to emissions
        final_S2 = initial_S2 * 0.8 # Assume S2 might deplete over time
        final_G = 10 # Assume some positive green capital builds up

        # Initial control variables at reasonable (but speculative) values
        initial_C = params.Y0 * 0.7 # Consumption is a fraction of initial output
        initial_N0 = params.N_0 * 0.5 # Half labor in final goods
        initial_LC = params.N_0 * 0.2 # Some labor in coal
        initial_N3 = params.N_0 * 0.1 # Some labor in green energy
        initial_Np = params.N_0 * 0.05 # Some labor in primary minerals
        initial_Ns = params.N_0 * 0.05 # Some labor in secondary minerals
        initial_F = params.S0_ghkt_oil_stock / T # Distribute oil consumption linearly
        initial_mp = params.Mp0_scenario / T # Distribute primary mineral linearly
        initial_ms = params.Ms0 / T # Distribute secondary mineral linearly

        # Initial scarcity rents and carbon tax (start small)
        initial_Mu_oil = 0.1
        initial_Mu_p = 0.1
        initial_Mu_s = 0.1
        initial_Lambda_s = params.gamma * params.Y0 * 1e-6 # Small initial tax

        # Populate the initial guess array
        for t in range(T):
            # Linearly interpolate state variables
            k_t_guess = initial_K0 + (final_K - initial_K0) * (t / T)
            s1_t_guess = initial_S1 + (final_S1 - initial_S1) * (t / T)
            s2_t_guess = initial_S2 + (final_S2 - initial_S2) * (t / T)
            oilstock_t_guess = initial_OilStock + (final_OilStock - initial_OilStock) * (t / T)
            mpstock_t_guess = initial_MpStock + (final_MpStock - initial_MpStock) * (t / T)
            msstock_t_guess = initial_MsStock + (final_MsStock - initial_MsStock) * (t / T)
            g_t_guess = initial_G + (final_G - initial_G) * (t / T)

            # Assign to 2D array
            initial_guess_2d[t, IDX_C] = initial_C
            initial_guess_2d[t, IDX_N0] = initial_N0
            initial_guess_2d[t, IDX_LC] = initial_LC
            initial_guess_2d[t, IDX_N3] = initial_N3
            initial_guess_2d[t, IDX_Np] = initial_Np
            initial_guess_2d[t, IDX_Ns] = initial_Ns
            initial_guess_2d[t, IDX_F] = initial_F
            initial_guess_2d[t, IDX_mp] = initial_mp
            initial_guess_2d[t, IDX_ms] = initial_ms
            initial_guess_2d[t, IDX_Mu_oil] = initial_Mu_oil
            initial_guess_2d[t, IDX_Mu_p] = initial_Mu_p
            initial_guess_2d[t, IDX_Mu_s] = initial_Mu_s
            initial_guess_2d[t, IDX_Lambda_s] = initial_Lambda_s
            initial_guess_2d[t, IDX_S1] = s1_t_guess
            initial_guess_2d[t, IDX_S2] = s2_t_guess
            initial_guess_2d[t, IDX_K] = k_t_guess
            initial_guess_2d[t, IDX_G] = g_t_guess
            initial_guess_2d[t, IDX_OilStock] = oilstock_t_guess
            initial_guess_2d[t, IDX_MpStock] = mpstock_t_guess
            initial_guess_2d[t, IDX_MsStock] = msstock_t_guess
            
            # Ensure non-negative and positive values where required for solver stability.
            # Add a small epsilon to avoid log(0) or division by zero during initial evaluations.
            initial_guess_2d[t, IDX_C] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_C])
            initial_guess_2d[t, IDX_N0] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_N0])
            initial_guess_2d[t, IDX_LC] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_LC])
            initial_guess_2d[t, IDX_N3] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_N3])
            initial_guess_2d[t, IDX_Np] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_Np])
            initial_guess_2d[t, IDX_Ns] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_Ns])
            initial_guess_2d[t, IDX_F] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_F])
            initial_guess_2d[t, IDX_mp] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_mp])
            initial_guess_2d[t, IDX_ms] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_ms])
            initial_guess_2d[t, IDX_Mu_oil] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_Mu_oil])
            initial_guess_2d[t, IDX_Mu_p] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_Mu_p])
            initial_guess_2d[t, IDX_Mu_s] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_Mu_s])
            initial_guess_2d[t, IDX_Lambda_s] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_Lambda_s])
            initial_guess_2d[t, IDX_S1] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_S1])
            initial_guess_2d[t, IDX_S2] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_S2])
            initial_guess_2d[t, IDX_K] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_K])
            initial_guess_2d[t, IDX_G] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_G])
            initial_guess_2d[t, IDX_OilStock] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_OilStock])
            initial_guess_2d[t, IDX_MpStock] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_MpStock])
            initial_guess_2d[t, IDX_MsStock] = np.maximum(epsilon_val, initial_guess_2d[t, IDX_MsStock])

            # Ensure labor shares sum to N_t (params.N_0) (approximately for initial guess)
            # This is a heuristic for the initial guess, not a strict equation.
            total_labor_guess = initial_guess_2d[t, IDX_N0] + initial_guess_2d[t, IDX_LC] + initial_guess_2d[t, IDX_N3] + \
                                initial_guess_2d[t, IDX_Np] + initial_guess_2d[t, IDX_Ns]
            
            if total_labor_guess > 0 and not np.isclose(total_labor_guess, params.N_0):
                scale_factor = params.N_0 / total_labor_guess
                initial_guess_2d[t, IDX_N0] *= scale_factor
                initial_guess_2d[t, IDX_LC] *= scale_factor
                initial_guess_2d[t, IDX_N3] *= scale_factor
                initial_guess_2d[t, IDX_Np] *= scale_factor
                initial_guess_2d[t, IDX_Ns] *= scale_factor
            else: # If total_labor_guess is 0 or very small for some reason, ensure a default non-zero value.
                if params.N_0 > 0:
                    initial_guess_2d[t, IDX_N0] = params.N_0 * 0.2
                    initial_guess_2d[t, IDX_LC] = params.N_0 * 0.2
                    initial_guess_2d[t, IDX_N3] = params.N_0 * 0.2
                    initial_guess_2d[t, IDX_Np] = params.N_0 * 0.2
                    initial_guess_2d[t, IDX_Ns] = params.N_0 * 0.2


        initial_guess_flat = initial_guess_2d.flatten()

    # --- Run the solver ---
    print(f"Attempting to solve for {T * NUM_VARS_PER_PERIOD} variables across {T} periods...")
    
    # Method 'lm' (Levenberg-Marquardt) is robust for non-linear least squares.
    # Method 'hybr' (hybrid Powell) is also good.
    # 'tol' is the tolerance for convergence.
    # 'max_nfev' is the maximum number of function evaluations for root.
    result = root(full_system_equations, initial_guess_flat, args=(params,), method='lm', tol=1e-8, options={'max_nfev': 5000})

    print(f"Solver result: {result.message}")
    print(f"Number of function evaluations: {result.nfev}")
    print(f"Success: {result.success}")
    print(f"Final residual norm: {np.linalg.norm(result.fun)}")

    if result.success:
        solution_reshaped = result.x.reshape(T, NUM_VARS_PER_PERIOD)
    else:
        solution_reshaped = initial_guess_2d # Return initial guess for inspection if no convergence
        print("\nSolver did not converge. Returning initial guess for inspection.")
        print(f"Final residuals (sum of squares): {np.sum(result.fun**2)}")
        print(f"Max absolute residual: {np.max(np.abs(result.fun))}")


    return result, solution_reshaped


# --- Main execution block to run the model ---
if __name__ == '__main__': # This ensures the code only runs when the script is executed directly
    # Instantiate parameters (already done globally but for clarity in main block)
    params = Parameters()

    # Run the solver
    solver_result, solution_variables = solve_model(params)

    if solver_result.success:
        print("\nModel solved successfully! Extracting and plotting results...")
        # Now you would extract the time series for each variable from solution_variables
        # and proceed to plot them, similar to the figures in GHKT and Chazel et al.

        # Example: Extracting Capital and Atmospheric Carbon
        K_series = solution_variables[:, IDX_K]
        S_total_series = solution_variables[:, IDX_S1] + solution_variables[:, IDX_S2]
        OilStock_series = solution_variables[:, IDX_OilStock]
        MpStock_series = solution_variables[:, IDX_MpStock]
        MsStock_series = solution_variables[:, IDX_MsStock]
        G_series = solution_variables[:, IDX_G]
        F_series = solution_variables[:, IDX_F]
        LC_series = solution_variables[:, IDX_LC]
        N3_series = solution_variables[:, IDX_N3]
        Np_series = solution_variables[:, IDX_Np]
        Ns_series = solution_variables[:, IDX_Ns]
        C_series = solution_variables[:, IDX_C]
        Y_series = np.zeros(params.T_horizon) # To be calculated
        Lambda_s_series = solution_variables[:, IDX_Lambda_s]

        # Recalculate Y_t for plotting, as it's not directly in solution_variables
        for t in range(params.T_horizon):
            N0_t = solution_variables[t, IDX_N0]
            LC_t = solution_variables[t, IDX_LC]
            N3_t = solution_variables[t, IDX_N3]
            F_t = solution_variables[t, IDX_F]
            S1_t = solution_variables[t, IDX_S1]
            S2_t = solution_variables[t, IDX_S2]
            K_t = solution_variables[t, IDX_K]
            G_t = solution_variables[t, IDX_G]

            # Use safe versions for calculation as well
            N0_t_safe = np.maximum(N0_t, epsilon_val)
            LC_t_safe = np.maximum(LC_t, epsilon_val)
            N3_t_safe = np.maximum(N3_t, epsilon_val)
            G_t_safe = np.maximum(G_t, epsilon_val)
            K_t_safe = np.maximum(K_t, epsilon_val)
            S_t_total_safe = np.maximum(S1_t + S2_t, epsilon_val)
            F_t_safe = np.maximum(F_t, epsilon_val)

            # Recalculate E3_t from solved G_t, N3_t
            A3_t_chazel = params.A3_0_chazel * (params.gA3_chazel**t)
            E3_t = green_energy_production_chazel_decadal(A3_t_chazel, params.gA3_chazel, t, N3_t_safe, G_t_safe, params)
            E3_t_safe = np.maximum(E3_t, epsilon_val)
            
            # Recalculate E_t_composite from solved F_t, LC_t, E3_t
            A2_t = params.A2_0 * (params.omega2**t)
            E_t_composite = energy_composite_decadal(F_t_safe, (A2_t * LC_t_safe), E3_t_safe, params)
            E_t_composite_safe = np.maximum(E_t_composite, epsilon_val)

            A0_t = 17887 # A0 is constant
            Y_series[t] = final_goods_output_decadal(K_t_safe, N0_t_safe, E_t_composite_safe, S_t_total_safe, A0_t, params, t)


        
        # Plotting (conceptual)
        time_years = np.arange(params.T_horizon) * params.time_step_length + 2010 # Years

        # Plot 1: Capital Stock
        plt.figure(figsize=(10, 6))
        plt.plot(time_years, K_series, label='Capital Stock (K_t)')
        plt.xlabel('Year')
        plt.ylabel('Capital (bn USD)')
        plt.title('Optimal Capital Stock Path')
        plt.legend()
        plt.grid(True)
        plt.show()

        # Plot 2: Total Atmospheric Carbon
        plt.figure(figsize=(10, 6))
        plt.plot(time_years, S_total_series, label='Total Atmospheric Carbon (S_t)')
        plt.xlabel('Year')
        plt.ylabel('Carbon (GtC)')
        plt.title('Optimal Atmospheric Carbon Path')
        plt.legend()
        plt.grid(True)
        plt.show()

        # Plot 3: Oil Stock
        plt.figure(figsize=(10, 6))
        plt.plot(time_years, OilStock_series, label='Oil Stock (R_oil,t)')
        plt.xlabel('Year')
        plt.ylabel('Oil Stock (GtC)')
        plt.title('Optimal Oil Stock Path')
        plt.legend()
        plt.grid(True)
        plt.show()

        # Plot 4: Primary Mineral Stock
        plt.figure(figsize=(10, 6))
        plt.plot(time_years, MpStock_series, label='Primary Mineral Stock (M_p,t)')
        plt.xlabel('Year')
        plt.ylabel('Mineral Stock (MtCu)')
        plt.title('Optimal Primary Mineral Stock Path')
        plt.legend()
        plt.grid(True)
        plt.show()

        # Plot 5: Secondary Mineral Stock
        plt.figure(figsize=(10, 6))
        plt.plot(time_years, MsStock_series, label='Secondary Mineral Stock (M_s,t)')
        plt.xlabel('Year')
        plt.ylabel('Mineral Stock (MtCu)')
        plt.title('Optimal Secondary Mineral Stock Path')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        # Plot 6: Green Capital Stock
        plt.figure(figsize=(10, 6))
        plt.plot(time_years, G_series, label='Green Capital Stock (G_t)')
        plt.xlabel('Year')
        plt.ylabel('Green Capital (MtCu)')
        plt.title('Optimal Green Capital Stock Path')
        plt.legend()
        plt.grid(True)
        plt.show()

        # Plot 7: Oil Extraction
        plt.figure(figsize=(10, 6))
        plt.plot(time_years, F_series, label='Oil Extraction (F_t)')
        plt.xlabel('Year')
        plt.ylabel('Oil (GtC/yr)')
        plt.title('Optimal Oil Extraction Path')
        plt.legend()
        plt.grid(True)
        plt.show()

        # Plot 8: Coal Extraction (A2_t * LC_t)
        Coal_Extraction_series = np.zeros(params.T_horizon)
        for t in range(params.T_horizon):
            A2_t = params.A2_0 * (params.omega2**t)
            Coal_Extraction_series[t] = A2_t * solution_variables[t, IDX_LC]

        plt.figure(figsize=(10, 6))
        plt.plot(time_years, Coal_Extraction_series, label='Coal Extraction (E2,t)')
        plt.xlabel('Year')
        plt.ylabel('Coal (GtC/yr)')
        plt.title('Optimal Coal Extraction Path')
        plt.legend()
        plt.grid(True)
        plt.show()

        # Plot 9: Green Energy Production (E3_t)
        E3_series = np.zeros(params.T_horizon)
        for t in range(params.T_horizon):
            N3_t = solution_variables[t, IDX_N3]
            G_t = solution_variables[t, IDX_G] # Green capital from solution
            A3_t_chazel = params.A3_0_chazel * (params.gA3_chazel**t)
            E3_series[t] = green_energy_production_chazel_decadal(A3_t_chazel, params.gA3_chazel, t, N3_t, G_t, params)

        plt.figure(figsize=(10, 6))
        plt.plot(time_years, E3_series, label='Green Energy Production (E3,t)')
        plt.xlabel('Year')
        plt.ylabel('Green Energy (Gtoe/yr)')
        plt.title('Optimal Green Energy Production Path')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        # Plot 10: Carbon Tax
        plt.figure(figsize=(10, 6))
        plt.plot(time_years, Lambda_s_series, label='Optimal Carbon Tax (Lambda_s,t)')
        plt.xlabel('Year')
        plt.ylabel('Tax ($/GtC)')
        plt.title('Optimal Carbon Tax Path')
        plt.legend()
        plt.grid(True)
        plt.show()

        # Plot 11: Total Output (Y_t)
        plt.figure(figsize=(10, 6))
        plt.plot(time_years, Y_series, label='Total Output (Y_t)')
        plt.xlabel('Year')
        plt.ylabel('Output (bn USD)')
        plt.title('Optimal Total Output Path')
        plt.legend()
        plt.grid(True)
        plt.show()

        # Plot 12: Total Consumption (C_t)
        plt.figure(figsize=(10, 6))
        plt.plot(time_years, C_series, label='Total Consumption (C_t)')
        plt.xlabel('Year')
        plt.ylabel('Consumption (bn USD)')
        plt.title('Optimal Total Consumption Path')
        plt.legend()
        plt.grid(True)
        plt.show()


    else:
        print("\nModel did not converge. Check initial guess and FOCs.")

Feasibility Check:
  Inequality constraints satisfied: True
  Equality constraints satisfied (|ceq| < 1e-6): False


C:\Users\7088256\AppData\Local\anaconda3\envs\chazel_model\Lib\site-packages\scipy\optimize\_slsqp_py.py:435: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
C:\Users\7088256\AppData\Local\anaconda3\envs\chazel_model\Lib\site-packages\scipy\optimize\_slsqp_py.py:439: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  g = append(wrapped_grad(x), 0.0)
C:\Users\7088256\AppData\Local\anaconda3\envs\chazel_model\Lib\site-packages\scipy\optimize\_slsqp_py.py:493: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  a_eq = vstack([con['jac'](x, *con['args'])
C:\Users\7088256\AppData\Local\anaconda3\envs\chazel_model\Lib\site-packages\scipy\optimize\_slsqp_py.py:499: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  a_ieq = vstack([con['jac'](x, *con['args'])
